# Project Proposal (**think of an interesting title later**)

**Group:** 3

**Members:** Maggie Cui, Tristan Heilbron, David Li, Ivy Nguyen

## Introduction

## Exploratory Data Analysis

### Loading required libraries

In [2]:
library(tidyverse)
library(tidymodels)
library(repr)
library(RColorBrewer)

In [3]:
# setting a seed so that the analysis is random but reproducible
set.seed(2005)

### Loading the dataset

In [5]:
data_column_names <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")

cleveland_data <- read_csv("data/processed.cleveland.data", col_names = data_column_names)
hungarian_data <- read_csv("data/processed.hungarian.data", col_names = data_column_names)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): ca, thal
dbl (12): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 294 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): trestbps, chol, fbs, restecg, thalach, exang, slope, ca, thal
dbl (5): age, sex, cp, oldpeak, num

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [27]:
glimpse(hungarian_data)

h <- hungarian_data |>
    select(trestbps) |> 
    distinct()
h

hungarian_data[hungarian_data == "?"] <- NA 

hungarian_data %>% 
    mutate (trestbps = as.numeric (trestbps),
            chol = as.numeric (chol),
            fbs = as.numeric (fbs), 
            restecg = as.numeric (restecg),
            thalach = as.numeric (thalach),
            exang = as.numeric (exang),
            slope = as.numeric (slope),
            ca = as.numeric (ca),
            thal = as.numeric (thal))

Rows: 294
Columns: 14
$ age      <dbl> 28, 29, 29, 30, 31, 32, 32, 32, 33, 34, 34, 34, 35, 35, 35, 3…
$ sex      <dbl> 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0…
$ cp       <dbl> 2, 2, 2, 1, 2, 2, 2, 2, 3, 2, 2, 2, 1, 4, 2, 2, 2, 3, 3, 3, 2…
$ trestbps <chr> "130", "120", "140", "170", "100", "105", "110", "125", "120"…
$ chol     <chr> "132", "243", NA, "237", "219", "198", "225", "254", "298", "…
$ fbs      <chr> "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "…
$ restecg  <chr> "2", "0", "0", "1", "1", "0", "0", "0", "0", "0", "1", "0", "…
$ thalach  <chr> "185", "160", "170", "170", "150", "165", "184", "155", "185"…
$ exang    <chr> "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "…
$ oldpeak  <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0…
$ slope    <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ ca       <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ thal     <chr> N

trestbps
<chr>
130
120
140
170
100
105
110
125
150


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
28,1,2,130,132,0,2,185,0,0,NA,NA,NA,0
29,1,2,120,243,0,0,160,0,0,NA,NA,NA,0
29,1,2,140,NA,0,0,170,0,0,NA,NA,NA,0
30,0,1,170,237,0,1,170,0,0,NA,NA,6,0
31,0,2,100,219,0,1,150,0,0,NA,NA,NA,0
32,0,2,105,198,0,0,165,0,0,NA,NA,NA,0
32,1,2,110,225,0,0,184,0,0,NA,NA,NA,0
32,1,2,125,254,0,0,155,0,0,NA,NA,NA,0
33,1,3,120,298,0,0,185,0,0,NA,NA,NA,0


## Methods

## Expected Outcomes and Significance

### Expected findings

### Impact of findings

### Future questions